In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token="</s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>"
)
model = AutoModelForCausalLM.from_pretrained(model_name)

from datasets import load_dataset

# 데이터셋 구조화
split_dict = {
    "train": "train[:9000]",
    "test": "train[9000:]",
}
dataset = load_dataset("csv", data_files="data/ChatBotData.csv", split=split_dict)

def tokenize(batch):
    return tokenizer([f"{ti}\n{te}" for ti, te in zip(batch["title"], batch["text"])])

# 토큰화
tokenized_dataset = dataset.map(
   tokenize,
    batched=True,
    num_proc=2,
    remove_columns=dataset["train"].column_names,
)

# 최대 길이로 그룹화
max_length = 512
def group_texts(batched_sample):
    sample = {k: v[0] for k, v in batched_sample.items()}

    if sample["input_ids"][-1] != tokenizer.eos_token_id:
        for k in sample.keys():
            sample[k].append(
                tokenizer.eos_token_id if k == "input_ids" else sample[k][-1]
            )

    result = {
        k: [v[i: i + max_length] for i in range(0, len(v), max_length)]
        for k, v in sample.items()
    }
    return result

grouped_dataset = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1,
    num_proc=2,
)
grouped_dataset["train"].column_names

Map (num_proc=2):   0%|          | 0/9000 [00:01<?, ? examples/s]


NameError: name 'tokenizer' is not defined

Map (num_proc=2):   0%|          | 0/9000 [00:01<?, ? examples/s]


NameError: name 'AutoTokenizer' is not defined